In [1]:
# %conda install langchain -c conda-forge

# Construyendo con LangChain
LangChain permite desarrollar aplicaciones que conectan fuentes externas de datos y cálculos a LLMs (modelos de lenguaje grandes). En este inicio rápido, recorreremos algunas formas diferentes de hacerlo. Comenzaremos con una cadena LLM simple, que se basa únicamente en la información en la plantilla de indicaciones para responder. A continuación, construiremos una cadena de recuperación, que obtiene datos de una base de datos independiente y los pasa a la plantilla de indicaciones. Luego, agregaremos un historial de chat para crear una cadena de recuperación de conversación. Esto permite interactuar de manera similar a un chat con este LLM, para que recuerde preguntas anteriores. Finalmente, construiremos un agente, que utiliza un LLM para determinar si necesita o no obtener datos para responder preguntas. Abordaremos estos temas a un nivel alto, ¡pero hay muchos detalles en todo esto! Enlazaremos a documentos relevantes.

## Cadena LLM
Para esta guía de inicio, proporcionaremos dos opciones: utilizar OpenAI (un modelo popular disponible a través de API) 

Primero, necesitaremos importar el paquete de integración LangChain x OpenAI.

In [2]:
# %pip install langchain-openai

In [3]:
# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

Una vez que hayas instalado e inicializado el LLM de tu elección, ¡podemos intentar usarlo! Preguntémosle qué es LangSmith: esto es algo que no estaba presente en los datos de entrenamiento, así que no debería tener una respuesta muy buena.

In [5]:
llm.invoke("¿Cómo puede ayudar Langsmith con las pruebas?")

AIMessage(content='Langsmith puede ayudar en las pruebas de varias maneras:\n\n1. Automatización de pruebas: Langsmith puede escribir scripts de pruebas automatizados para ejecutar pruebas repetitivas y garantizar la consistencia en los resultados. Esto ayuda a ahorrar tiempo y esfuerzo en las pruebas manuales.\n\n2. Generación automática de datos de prueba: Langsmith puede generar datos de prueba automáticamente, lo que facilita la creación de conjuntos de prueba exhaustivos y diversos. Esto ayuda a descubrir posibles problemas o errores en el software.\n\n3. Detección de errores y seguimiento de problemas: Langsmith puede monitorear y registrar los errores encontrados durante las pruebas. Esto ayuda a identificar rápidamente los problemas y seguir su resolución.\n\n4. Análisis y visualización de resultados de pruebas: Langsmith puede analizar los resultados de las pruebas y presentarlos de manera visualmente atractiva. Esto facilita la comprensión de los resultados de las pruebas y l

También podemos guiar su respuesta con una plantilla de indicaciones. Las plantillas de indicaciones se utilizan para convertir la entrada de usuario sin procesar en una entrada mejor para el LLM.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un escritor de documentación técnica de clase mundial."),
    ("user", "{entrada}")
])

Ahora podemos combinar estas en una cadena LLM simple:

In [7]:
chain = prompt | llm 

Ahora podemos invocarlo y hacer la misma pregunta. Aún no conocerá la respuesta, pero debería responder en un tono más adecuado para un escritor técnico.

In [8]:
chain.invoke({"entrada": "¿Cómo puede ayudar Langsmith con las pruebas?"})

AIMessage(content='Langsmith puede ayudar con las pruebas de diferentes maneras:\n\n1. Automatización de pruebas: Langsmith puede ayudar a desarrollar scripts de pruebas automatizadas para acelerar el proceso de prueba y garantizar la consistencia de los resultados. Esto puede incluir pruebas de unidad, pruebas de integración, pruebas de regresión, etc.\n\n2. Generación de datos de prueba: Langsmith puede generar datos de prueba realistas y relevantes para las pruebas. Esto puede incluir datos de prueba aleatorios, datos de prueba basados en patrones o datos de prueba específicos para cubrir casos de uso particulares.\n\n3. Creación de casos de prueba: Langsmith puede ayudar a crear casos de prueba detallados y exhaustivos para garantizar que todas las funcionalidades y requisitos del sistema se prueben adecuadamente. Esto incluye la identificación de escenarios de prueba, la definición de pasos de prueba y la especificación de resultados esperados.\n\n4. Documentación de pruebas: Lang

La salida de un modelo de chat (y, por lo tanto, de esta cadena) es un mensaje. Sin embargo, a menudo es mucho más conveniente trabajar con cadenas. Añadamos un simple analizador de salida para convertir el mensaje de chat en una cadena.

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Ahora podemos agregar esto a la cadena anterior:

In [10]:
chain = prompt | llm | output_parser

Ahora podemos invocarlo y hacer la misma pregunta. La respuesta ahora será una cadena (en lugar de un ChatMessage).

In [11]:
chain.invoke({"entrada": "¿Cómo puede ayudar Langsmith con las pruebas?"})

'Langsmith puede ayudar en las pruebas de varias maneras:\n\n1. Documentación de casos de prueba: Como escritor de documentación técnica, puedo ayudar a crear una documentación detallada de los casos de prueba. Esto incluye describir los pasos necesarios para ejecutar las pruebas, los datos de entrada requeridos y los resultados esperados.\n\n2. Creación de guías de prueba: Puedo desarrollar guías de prueba que proporcionen instrucciones paso a paso sobre cómo ejecutar las pruebas correctamente. Estas guías pueden incluir capturas de pantalla, ejemplos de código y explicaciones detalladas para facilitar la comprensión y ejecución de las pruebas.\n\n3. Generación de informes de prueba: Puedo ayudar a generar informes de prueba claros y concisos que muestren los resultados de las pruebas realizadas. Estos informes pueden incluir métricas de rendimiento, errores encontrados, cobertura de código y cualquier otra información relevante.\n\n4. Revisión de documentación existente: Si ya tienes

## Profundizando​
Ahora hemos configurado con éxito una cadena LLM básica. Solo hemos explorado los conceptos básicos de indicaciones, modelos y analizadores de salida; para obtener una comprensión más profunda de todo lo mencionado aquí, consulta esta sección de la documentación.

## Cadena de Recuperación
Para responder adecuadamente a la pregunta original ("¿cómo puede ayudar LangSmith con las pruebas?"), necesitamos proporcionar contexto adicional al LLM. Podemos hacer esto mediante la recuperación. La recuperación es útil cuando tienes demasiados datos para pasar al LLM directamente. Luego puedes usar un recuperador para buscar solo las piezas más relevantes y pasarlas.

En este proceso, buscaremos documentos relevantes desde un Recuperador y luego los pasaremos a la indicación. Un Recuperador puede basarse en cualquier cosa, como una tabla SQL, internet, etc., pero en este caso poblaremos una tienda de vectores y la utilizaremos como un recuperador. Para obtener más información sobre las tiendas de vectores, consulta esta documentación.

Primero, necesitamos cargar los datos que queremos indexar. Para hacer esto, utilizaremos el WebBaseLoader. Esto requiere instalar BeautifulSoup:

In [12]:
# %conda install beautifulsoup4

Después de eso, podemos importar y usar WebBaseLoader.

In [13]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

A continuación, necesitamos indexarlo en una tienda de vectores. Esto requiere algunos componentes, en particular, un modelo de incrustación y una tienda de vectores.

En cuanto a los modelos de incrustación, proporcionamos nuevamente ejemplos de acceso a través de OpenAI o mediante modelos locales.


Asegúrate de tener instalado el paquete `langchain_openai` y las variables de entorno adecuadas configuradas (son las mismas que se necesitan para el LLM).

In [14]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

Ahora podemos utilizar este modelo de incrustación para ingresar documentos en una tienda de vectores. Utilizaremos una tienda de vectores local y sencilla, FAISS, por simplicidad.

Primero, necesitamos instalar los paquetes necesarios para ello:

In [15]:
# %conda install faiss-cpu

Luego podemos construir nuestro índice:

In [16]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Ahora que tenemos estos datos indexados en una tienda de vectores, crearemos una cadena de recuperación. Esta cadena tomará una pregunta entrante, buscará documentos relevantes y luego pasará esos documentos junto con la pregunta original a un LLM y le pedirá que responda la pregunta original.

Primero, configuremos la cadena que toma una pregunta y los documentos recuperados y genera una respuesta.

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Responda la siguiente pregunta basándose únicamente en el contexto proporcionado:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [18]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "¿Cómo puede ayudar Langsmith con las pruebas?",
    "context": [Document(page_content="Langsmith puede permitirle visualizar los resultados de las pruebas")]
})

'Langsmith puede ayudar a visualizar los resultados de las pruebas.'

Sin embargo, queremos que los documentos provengan primero del recuperador que acabamos de configurar. De esta manera, para una pregunta dada, podemos utilizar el recuperador para seleccionar dinámicamente los documentos más relevantes y pasarlos.

In [19]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Ahora podemos invocar esta cadena. Esto devuelve un diccionario: la respuesta del LLM se encuentra en la clave "answer".

In [20]:
response = retrieval_chain.invoke({"input":"¿Cómo puede ayudar Langsmith con las pruebas?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith puede ayudar con las pruebas al proporcionar una forma fácil de visualizar los resultados de las llamadas a modelos de lenguaje (LLMs) y al permitir la modificación de las entradas de prueba en un "playground" para observar los cambios en la salida. También simplifica la carga de conjuntos de datos para probar cambios en los prompts o cadenas. Además, LangSmith ofrece evaluadores automáticos para analizar los resultados de las pruebas y permite la revisión y anotación manual de las ejecuciones a través de colas de anotaciones, lo que permite una revisión humana de calidad y confiabilidad.


## Cadena de Recuperación de Conversación
La cadena que hemos creado hasta ahora solo puede responder preguntas individuales. Uno de los principales tipos de aplicaciones de LLM que las personas están desarrollando son los chatbots. Entonces, ¿cómo convertimos esta cadena en una que pueda responder preguntas de seguimiento?

Todavía podemos utilizar la función create_retrieval_chain, pero necesitamos cambiar dos cosas:

1. El método de recuperación ahora no solo debe funcionar en la entrada más reciente, sino que debe tener en cuenta toda la historia.
2. La cadena final del LLM también debe tener en cuenta toda la historia.

Actualización de la Recuperación

Para actualizar la recuperación, crearemos una nueva cadena. Esta cadena tomará la entrada más reciente (input) y el historial de la conversación (chat_history) y utilizará un LLM para generar una consulta de búsqueda.

In [21]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Dada la conversación anterior, genere una consulta de búsqueda para buscar información relevante para la conversación.")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

Podemos probar esto pasando una instancia en la que el usuario hace una pregunta de seguimiento.

In [22]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="¿Puede LangSmith ayudarme a probar mis solicitudes de LLM?"), AIMessage(content="¡Si!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Dime como"
})

[Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is achieved by setting environment variables, which we establish whenever we launch a virtual environment or open our bash shell and leave them set. The same 

Deberías ver que esto devuelve documentos sobre pruebas en LangSmith. Esto se debe a que el LLM generó una nueva consulta, combinando el historial del chat con la pregunta de seguimiento.

Ahora que tenemos este nuevo recuperador, podemos crear una nueva cadena para continuar la conversación teniendo en cuenta estos documentos recuperados.

In [23]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Responda las preguntas del usuario según el siguiente contexto:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

Ahora a probar

In [24]:
chat_history = [HumanMessage(content="¿Puede LangSmith ayudarme a probar mis solicitudes de LLM?"), AIMessage(content="¡SI!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Dime como"
})

{'chat_history': [HumanMessage(content='¿Puede LangSmith ayudarme a probar mis solicitudes de LLM?'),
  AIMessage(content='¡SI!')],
 'input': 'Dime como',
 'context': [Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python si

Podemos ver que esto da una respuesta coherente: ¡hemos convertido con éxito nuestra cadena de recuperación en un chatbot!

## Agente
Hasta ahora, hemos creado ejemplos de cadenas, donde cada paso se conoce de antemano. Lo último que crearemos es un agente, donde el LLM decide qué pasos tomar.

NOTA: para este ejemplo, solo mostraremos cómo crear un agente utilizando modelos de OpenAI, ya que los modelos locales aún no son lo suficientemente confiables.

Una de las primeras cosas que debes hacer al construir un agente es decidir a qué herramientas debería tener acceso. Para este ejemplo, daremos al agente acceso a dos herramientas:

1. El recuperador que acabamos de crear. Esto le permitirá responder fácilmente preguntas sobre LangSmith.
2. Una herramienta de búsqueda. Esto le permitirá responder fácilmente preguntas que requieran información actualizada.

Primero, configuremos una herramienta para el recuperador que acabamos de crear:

In [25]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Buscar información sobre LangSmith. ¡Para cualquier pregunta sobre LangSmith, debes utilizar esta herramienta!",
)

La herramienta de búsqueda que utilizaremos es Tavily. Esto requerirá una clave API (tienen un generoso nivel gratuito). Después de crearlo en su plataforma, debes configurarlo como una variable de entorno:

Actualizamos el .env con la clave de tavily

In [26]:
import dotenv

dotenv.load_dotenv()

True

In [27]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

Ahora podemos crear una lista de las herramientas con las que queremos trabajar:

In [28]:
tools = [retriever_tool, search]

Ahora que tenemos las herramientas, podemos crear un agente para usarlas. Repasaremos esto rápidamente; para profundizar en lo que está sucediendo exactamente, consulte la documentación de introducción del agente. 

Instale primero el HUB langchain

In [29]:
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


Ahora podemos usarlo para obtener un mensaje predefinido.

In [30]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

¡Ahora podemos invocar al agente y ver cómo responde! Podemos hacerle preguntas sobre LangSmith:

In [31]:
agent_executor.invoke({"input": "¿Cómo puede ayudar Langsmith con las pruebas?"})



> Entering new AgentExecutor chain...
LangSmith puede ayudar con las pruebas de varias maneras:

1. Generación automática de casos de prueba: LangSmith puede generar automáticamente casos de prueba para diferentes escenarios y condiciones. Esto ahorra tiempo y esfuerzo al crear casos de prueba manualmente.

2. Ejecución de pruebas automatizadas: LangSmith puede ejecutar pruebas automatizadas para verificar el funcionamiento correcto de una aplicación o sistema. Esto incluye la ejecución de pruebas unitarias, pruebas de integración y pruebas de regresión.

3. Análisis de cobertura de código: LangSmith puede analizar la cobertura de código de las pruebas para identificar áreas que no están siendo probadas adecuadamente. Esto ayuda a garantizar una cobertura completa y efectiva de las pruebas.

4. Generación de informes de pruebas: LangSmith puede generar informes detallados sobre los resultados de las pruebas, incluyendo información sobre los casos de prueba ejecutados, los errores enc

{'input': '¿Cómo puede ayudar Langsmith con las pruebas?',
 'output': 'LangSmith puede ayudar con las pruebas de varias maneras:\n\n1. Generación automática de casos de prueba: LangSmith puede generar automáticamente casos de prueba para diferentes escenarios y condiciones. Esto ahorra tiempo y esfuerzo al crear casos de prueba manualmente.\n\n2. Ejecución de pruebas automatizadas: LangSmith puede ejecutar pruebas automatizadas para verificar el funcionamiento correcto de una aplicación o sistema. Esto incluye la ejecución de pruebas unitarias, pruebas de integración y pruebas de regresión.\n\n3. Análisis de cobertura de código: LangSmith puede analizar la cobertura de código de las pruebas para identificar áreas que no están siendo probadas adecuadamente. Esto ayuda a garantizar una cobertura completa y efectiva de las pruebas.\n\n4. Generación de informes de pruebas: LangSmith puede generar informes detallados sobre los resultados de las pruebas, incluyendo información sobre los caso

Podemos preguntarle sobre el clima:

In [32]:
agent_executor.invoke({"input": "¿Cuál es el clima en NY?"})



> Entering new AgentExecutor chain...



Invoking: `tavily_search_results_json` with `{'query': 'clima en NY'}`


[{'url': 'https://www.expreso.ec/actualidad/mundo/nueva-york-alista-tormenta-dejar-20-centimetros-nieve-189453.html', 'content': 'Clima extremo El gran negocio del nuevo aeropuerto de Guayaquil 11 febrero, 2024  Nueva York: alerta por tormenta invernal con lluvias y potenciales inundaciones  Mundo Nueva York se alista para tormenta que puede dejar hasta 20 centímetros de nieve  El cabildo local ordenó clases virtuales para este martes 13 de febrero de 2024. Se esperan fuertes vientos4 hours ago — 4 hours agoNueva York se alista para tormenta que puede dejar hasta 20 centímetros de nieve. El cabildo local ordenó clases virtuales para este martes 13\xa0...'}]Según los resultados de búsqueda, en Nueva York se espera una tormenta invernal con lluvias y potenciales inundaciones. También se espera que la tormenta deje hasta 20 centímetros de nieve. El cabildo local ha ordenado clases virtuales para el martes 13 de febr

{'input': '¿Cuál es el clima en NY?',
 'output': 'Según los resultados de búsqueda, en Nueva York se espera una tormenta invernal con lluvias y potenciales inundaciones. También se espera que la tormenta deje hasta 20 centímetros de nieve. El cabildo local ha ordenado clases virtuales para el martes 13 de febrero de 2024 debido a esta situación.'}

In [33]:
chat_history = [HumanMessage(content="¿Puede LangSmith ayudarme a probar mis solicitudes de LLM?"), AIMessage(content="¡Si!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Dime como"
})



> Entering new AgentExecutor chain...


Para probar tus solicitudes de LLM (Lenguaje de Modelado de Lenguajes) con LangSmith, puedes seguir estos pasos:

1. Formula tu solicitud de LLM en un formato compatible con LangSmith. Puedes utilizar el lenguaje de programación que prefieras para escribir tu solicitud, como Python, JavaScript, o cualquier otro lenguaje compatible con la API de LangSmith.

2. Utiliza la función `langsmith_search` de LangSmith para enviar tu solicitud y obtener los resultados. Esta función acepta un objeto con la propiedad `query`, que debe ser tu solicitud de LLM en formato de texto.

3. Analiza los resultados devueltos por la función `langsmith_search`. Puedes acceder a la información relevante de los resultados para evaluar la precisión y calidad de tu solicitud de LLM.

Aquí tienes un ejemplo de cómo podrías utilizar la función `langsmith_search` en JavaScript:

```javascript
const query = "Mi solicitud de LLM";
const results = langsmith_search({ query });

// Analiza los resultados devueltos
consol

{'chat_history': [HumanMessage(content='¿Puede LangSmith ayudarme a probar mis solicitudes de LLM?'),
  AIMessage(content='¡Si!')],
 'input': 'Dime como',
 'output': 'Para probar tus solicitudes de LLM (Lenguaje de Modelado de Lenguajes) con LangSmith, puedes seguir estos pasos:\n\n1. Formula tu solicitud de LLM en un formato compatible con LangSmith. Puedes utilizar el lenguaje de programación que prefieras para escribir tu solicitud, como Python, JavaScript, o cualquier otro lenguaje compatible con la API de LangSmith.\n\n2. Utiliza la función `langsmith_search` de LangSmith para enviar tu solicitud y obtener los resultados. Esta función acepta un objeto con la propiedad `query`, que debe ser tu solicitud de LLM en formato de texto.\n\n3. Analiza los resultados devueltos por la función `langsmith_search`. Puedes acceder a la información relevante de los resultados para evaluar la precisión y calidad de tu solicitud de LLM.\n\nAquí tienes un ejemplo de cómo podrías utilizar la función

## Sirviendo con LangServe
Ahora que hemos construido una aplicación, necesitamos ponerla en funcionamiento. Ahí es donde entra LangServe. LangServe ayuda a los desarrolladores a implementar cadenas de LangChain como una API REST. No es necesario utilizar LangServe para usar LangChain, pero en esta guía mostraremos cómo puedes implementar tu aplicación con LangServe.

Mientras que la primera parte de esta guía estaba destinada a ejecutarse en un cuaderno Jupyter, ahora saldremos de eso. Crearemos un archivo Python y luego interactuaremos con él desde la línea de comandos.

In [34]:
# %pip install "langserve[all]"

Note: you may need to restart the kernel to use updated packages.


## Servidor 
Para crear un servidor para nuestra aplicación, crearemos un archivo [serve.py](http://localhost:8889/lab/tree/Documentos/GitClone/CookBook/LangChain/serve.py). 
Esto contendrá nuestra lógica para servir nuestra aplicación. 
Consta de tres cosas: 
1. La definición de nuestra cadena que acabamos de crear arriba
2. Nuestra aplicación FastAPI
3. Una definición de una ruta desde la cual servir a la cadena, que se hace con langserve.add_routes

deberíamos ver nuestra [cadena](http://localhost:8889/lab/tree/Documentos/GitClone/CookBook/LangChain/serve.py) siendo atendida en localhost:8000.

## Playground 
Cada servicio LangServe viene con una interfaz de usuario integrada simple para configurar e invocar la aplicación con salida de transmisión y visibilidad de los pasos intermedios. 
Dirígete a http://localhost:8000/agent/playground/ ¡para probarlo! Pase la misma pregunta que antes: "¿Cómo puede ayudar Langsmith con las pruebas?" - y debería responder igual que antes.

## Cliente 
Ahora configuremos un cliente para interactuar mediante programación con nuestro servicio. Podemos hacer esto fácilmente con [langserve.RemoteRunnable](/docs/langserve#client) Al usar esto, podemos interactuar con la cadena servida como si se estuviera ejecutando en el lado del cliente.

In [36]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")
remote_chain.invoke({
    "input": "¿Cómo puede ayudar Langsmith con las pruebas?",
    "chat_history": []  # Providing an empty list as this is the first call
})

{'output': 'LangSmith puede ayudar con las pruebas de varias maneras:\n\n1. Generación automática de casos de prueba: LangSmith puede generar automáticamente casos de prueba para diferentes escenarios y condiciones. Esto puede ahorrar tiempo y esfuerzo en la creación manual de casos de prueba.\n\n2. Ejecución de pruebas automatizadas: LangSmith puede ejecutar pruebas automatizadas para verificar el funcionamiento correcto de un sistema o aplicación. Esto puede incluir pruebas de regresión, pruebas de integración y pruebas de rendimiento.\n\n3. Análisis de resultados de pruebas: LangSmith puede analizar los resultados de las pruebas y proporcionar informes detallados sobre los errores encontrados, la cobertura de código y otros aspectos importantes de las pruebas.\n\n4. Optimización de pruebas: LangSmith puede ayudar a optimizar las pruebas identificando áreas de código que no están siendo probadas adecuadamente y sugiriendo mejoras en la estrategia de pruebas.\n\n5. Integración con her